# 配置类

In [162]:
class Config(object):
    poetry_file = "data/poetry.txt"
    weight_file = "data/poetry_model.h5"
    # 根据前六个字预测第七个字
    max_len = 6
    batch_size = 512
    learning_rate = 0.001

# 语料预处理

In [163]:
puncs = [']', '[', '（', '）', '{', '}', '：', '《', '》']

def preprocess_file(Config):
    file_content = ""
    with open(Config.poetry_file, "r", encoding="utf-8") as f:
        for line in f:
            for char in puncs:
                line.replace(char, "")
            file_content += line.strip() + "]"
    
    words = sorted(list(file_content))
    counted_words = {}
    for word in words:
        if word in counted_words:
            counted_words[word] += 1
        else:
            counted_words[word] = 1
            
    # 去掉低频字
    erase = []
    for key in counted_words:
        if counted_words[key] <= 2:
            erase.append(key)        
    for key in erase:
        del counted_words[key]
        
    del counted_words["]"]
    
    wordPairs = sorted(counted_words.items(), key=lambda x: -x[1])
    words, _ = zip(*wordPairs)
    # word到id的映射
    word2num = dict((c, i + 1)for i, c in enumerate(words))
    num2word = dict((i, c) for i, c in enumerate(words))
    word2numF = lambda x: word2num.get(x, 0)
    return word2numF, num2word, words, file_content

# 构建模型

In [164]:
import os
from keras.layers import Input, Embedding, Bidirectional, Dropout, Flatten, Dense, GRU
from keras.models import Model, load_model
from keras.optimizers import Adam
import numpy as np
import keras
from keras.callbacks import LambdaCallback
import random

In [165]:
class PoetryModel(object):
    def __init__(self, config):
        self.model = None
        self.do_train = True
        self.loaded_model = False
        self.config = config

        # 文件预处理
        self.word2numF, self.num2word, self.words, self.files_content = preprocess_file(self.config)
        if os.path.exists(self.config.weight_file):
            self.model = load_model(self.config.weight_file)
            self.model.summary()
        else:
            self.train()
        self.do_train = False
        self.loaded_model = True
    
    def build_model(self):
        """建立模型"""
        input_tensor = Input(shape=(self.config.max_len,))
        embedd = Embedding(len(self.num2word)+1, 300, input_length=self.config.max_len)(input_tensor)
        lstm = Bidirectional(GRU(128, return_sequences=True))(embedd)
        dropout = Dropout(0.6)(lstm)
        lstm = Bidirectional(GRU(128, return_sequences=True))(embedd)
        dropout = Dropout(0.6)(lstm)
        flatten = Flatten()(lstm)
        dense = Dense(len(self.words), activation='softmax')(flatten)
        self.model = Model(inputs=input_tensor, outputs=dense)
        optimizer = Adam(lr=self.config.learning_rate)
        self.model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    def sample(self, preds, temperature=1.0):
        """
        当temperature=1.0时，模型输出正常
        当temperature=0.5时，模型输出比较open
        当temperature=1.5时，模型输出比较保守
        """
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)
    
    def generate_sample_result(self, epoch, logs):
        print("\n==================Epoch {}=====================".format(epoch))
        for diversity in [0.5, 1.0, 1.5]:
            print("------------Diversity {}--------------".format(diversity))
            start_index = random.randint(0, len(self.files_content) - self.config.max_len - 1)
            generated = ''
            sentence = self.files_content[start_index: start_index + self.config.max_len]
            generated += sentence
            for i in range(20):
                x_pred = np.zeros((1, self.config.max_len))
                for t, char in enumerate(sentence[-6:]):
                    x_pred[0, t] = self.word2numF(char)

                preds = self.model.predict(x_pred, verbose=0)[0]
                next_index = self.sample(preds, diversity)
                next_char = self.num2word[next_index]
                generated += next_char
                sentence = sentence + next_char
            print(sentence)
        
    def predict(self, text):
        if not self.loaded_model:
            return
        with open(self.config.poetry_file, 'r', encoding='utf-8') as f:
            file_list = f.readlines()
        random_line = random.choice(file_list)
        # 如果给的text不到四个字，则随机补全
        if not text or len(text) != 4:
            for _ in range(4 - len(text)):
                random_str_index = random.randrange(0, len(self.words))
                text += self.num2word.get(random_str_index) if self.num2word.get(random_str_index) not in [',', '。',
                                                                                                           '，'] else self.num2word.get(
                    random_str_index + 1)
        seed = random_line[-(self.config.max_len):-1]
        res = ''
        seed = 'c' + seed
        for c in text:
            seed = seed[1:] + c
            for j in range(5):
                x_pred = np.zeros((1, self.config.max_len))
                for t, char in enumerate(seed):
                    x_pred[0, t] = self.word2numF(char)
                preds = self.model.predict(x_pred, verbose=0)[0]
                next_index = self.sample(preds, 1.0)
                next_char = self.num2word[next_index]
                seed = seed[1:] + next_char
            res += seed
        return res
        
    
    def data_generator(self):
        """生成数据，提供给模型训练时使用"""
        i = 0
        while 1:
            x = self.files_content[i: i + self.config.max_len]
            y = self.files_content[i + self.config.max_len]
            puncs = [']', '[', '（', '）', '{', '}', '：', '《', '》', ':']
            if len([i for i in puncs if i in x]) != 0:
                i += 1
                continue
            if len([i for i in puncs if i in y]) != 0:
                i += 1
                continue
            y_vec = np.zeros(
                shape=(1, len(self.words)),
                dtype=np.bool
            )
            y_vec[0, self.word2numF(y)] = 1.0
            x_vec = np.zeros(
                shape=(1, self.config.max_len),
                dtype=np.int32
            )
            for t, char in enumerate(x):
                x_vec[0, t] = self.word2numF(char)
            yield x_vec, y_vec
            i += 1
    
    def train(self):
        #number_of_epoch = len(self.files_content) // self.config.batch_size
        number_of_epoch = 10
        if not self.model:
            self.build_model()
        self.model.summary()
        self.model.fit_generator(
            generator=self.data_generator(),
            verbose=True,
            steps_per_epoch=self.config.batch_size,
            epochs=number_of_epoch,
            callbacks=[
                keras.callbacks.ModelCheckpoint(self.config.weight_file, save_weights_only=False),
                LambdaCallback(on_epoch_end=self.generate_sample_result)
            ]
        )

# 开始训练模型

In [166]:
model = PoetryModel(Config)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 6, 300)            1798800   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 6, 256)            329472    
_________________________________________________________________
flatten_7 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 5995)              9214315   
Total params: 11,342,587
Trainable params: 11,342,587
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
512/512 [==============================] - 86s 168ms/step - loss: 7.4454 - acc: 0.1504

==================Epoch 0======

# 模型作诗

In [167]:
input = "小船"
sentence = model.predict(input)
print(sentence)

小干谔醆椹。船电欺廒吾。旱勋能劳亲。忌贷滩蝶换。


In [168]:
print(model.predict("小雨"))

小平佛獠近。雨见默关祛。越乖得当盖。怙忠咋未春。
